In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import time
import pickle

In [2]:
recordE=pd.read_csv("../input/categories/recordE.csv")

In [3]:
recordE.head()

,document_id,status,document_url,media_type,media_link,relative_media_path,is_media_downloaded,transcription_text,transcription_text_relative_path,from_time,to_time,project_id,Category,Unnamed: 13,Unnamed: 14
0,NMAAHC-007675740_00017,Complete,https://transcription.si.edu/transcribe/15636/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/15636/image/NMAAHC-007675740_00017.jpg,1,Abstract of Disbursements on a/c of Schools.\r...,/15636/transcriptions/NMAAHC-007675740_00017.txt,\N,\N,15636,Records - Education,NaN,NaN
1,NMAAHC-007675740_00018,Complete,https://transcription.si.edu/transcribe/15636/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/15636/image/NMAAHC-007675740_00018.jpg,1,Disbursments on account of Colored Schools for...,/15636/transcriptions/NMAAHC-007675740_00018.txt,\N,\N,15636,Records - Education,NaN,NaN
2,NMAAHC-007675740_00019,Complete,https://transcription.si.edu/transcribe/15636/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/15636/image/NMAAHC-007675740_00019.jpg,1,Disbursements on account of Colored Schools Ja...,/15636/transcriptions/NMAAHC-007675740_00019.txt,\N,\N,15636,Records - Education,NaN,NaN
3,NMAAHC-007675740_00020,Complete,https://transcription.si.edu/transcribe/15636/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/15636/image/NMAAHC-007675740_00020.jpg,1,School Expenses for Month of February /67,/15636/transcriptions/NMAAHC-007675740_00020.txt,\N,\N,15636,Records - Education,NaN,NaN
4,NMAAHC-007675740_00021,Complete,https://transcription.si.edu/transcribe/15636/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/15636/image/NMAAHC-007675740_00021.jpg,1,"James F. Carlen & Co. 5 keys, Nails, School Ho...",/15636/transcriptions/NMAAHC-007675740_00021.txt,\N,\N,15636,Records - Education,NaN,NaN


In [4]:
recordE.shape

(13487, 15)

In [5]:
recordE_text=recordE["transcription_text"]

In [6]:
print(recordE_text[0])

Abstract of Disbursements on a/c of Schools.
From Jany/67 to July, 1868.


In [7]:
print(recordE_text[10])

Paid in April on Acct of Schools.

[[2 column table - No headings]]
Thomas Evans 22nd and I Sts (repairs of Water Pipes) | 6?.05 |

Philip Brown. 17the and I [[ditto for Sts (repairs]] do of Roofs | 5.00 |

Uhlinger.(Phil.d) School Desks | 260.40 |

A Bowen  Rent | 6.00 |

School. Cambridge Park Map. | 180.64 |

Fencing 7th ave Boundary | 13.38 |

School Martinsburg Va. | 5.00 |

[[ditto for School]] Shepherdstown Va. | 3.00 |
[[ditto for School]] Charlestown Va. | 5.00|

Wallace Building Boundary St | 60.00 |

John Kimball Supt. Schools | 146.67 |

R Young 17th and O | 15.00 |

[[tally line]]
[[underlined]] $769.14 [/underlined]]






In [8]:
print(recordE_text[100])

[[preprinted]] 138 [[/preprinted]]

Denton Md. School House (continued from Page 74)

[[table 4 columns]]
[[Date | Item | Unit Price | Total]]

1867 | Per amount bro't forward |  | 524.90
 
  | Per bill of Chappell & Leary |  | 
Nov 27" | For 35 Galls. Linseed Oil @ $1.25 | 43.75 | 
  | 1 Gall. Turpentine | 1.37 | 
  | 150 lbs Brandon Red @ .5 | 7.50 | 
  | 75 lbs Brandon Yellow @ .5 | 3.75 | 
  | 100 lbs N.Y. Lead | 12.00 | 
  | 15 lbs Chrome Green (oil) @ .30 | 4.50 | 
  | 8 lbs Litharge @ 18 | 1.44 | 
  | 1 lb Lampblack | .12 | 
  | 8 lbs Putty @ .7 | .56 | 74.99
 
  | Per bill of Millard F. Baker |  | 
Oct 29" | Services as Carpenter from Oct 1" to 28"/67 being 24 3/4 days @ $1.75 per day | 43.31 | 43.31

  | Per bill of J. Smith |  | 
Oct 29 | Services as Laborer from Oct 1" to 5"/67 being 4 days @ $1.50 per day | 6.00 | 6.00

  | Per bill of Wm. H. Baker |  | 
Oct 29" | For services as Carpenter from Oct 1" to 28"/67 being 19 1/4 days @ $2.30 per day | 44.28 | 44.28

  | Per bill

In [9]:
print(recordE_text[5000])

Feb 67
School Report
of James P. Butter L.A.C.

County of Walker

For the Month of November 1867

Forwarded November 31st 1867
Received O.S Dec 8th 1867


In [10]:
print(recordE_text[7000])

1. Amount of Tuition paid during the month, 
$3.50/100. d

2. Remarks. Every thing in Our Schools, are progressive, and encourageing.


In [11]:
print(recordE_text[10000])

Parish of Assumptions
This Parish is divided into 20 School districts one of which is deemed unworth of a school. There an therefore but 19.

School No 1.
Miss Mary Newman  Teacher
This school is located on the plantation of Elie Landry in the neighborhood of Labadieville & near the lower side of the parish. on the Right Bank of Bay on Lafourche. In this district there are about 60 children between the age of 5 & 14 yrs. about 50 only attend the school.

50 children

Schools Nos 2 & 3 Have not as yet been opened. There are in the two districts between the age of 5 & 14

120 Cold. children

See Remarks acc Report.
229-1000


In [12]:
print(recordE_text[13000])

April 2nd
Mrs. Amelia Hirscheimer
You are appointed Teacher in the School No [[1?]] Parish of Ascension.
You will commence your duties immediately.
E M Wheelock
Lieut & Secty



April 3rd
Miss Mattie M. Fitts
You are appointed assistant in School No [[12?]] City of New Orleans.
You will commence your duties without delay.
E M Wheelock
Lieut & Secty.

5


In [13]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()
stop_words.remove("of")
punct='!"#$%&\'()*+-/<=>?@[\\]^_`{|}~'
def process(s):
    s= re.sub("([\[]).*?([\]])"," ",s)
    for p in punct:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    return s
for i in tqdm(range(len(recordE_text))):
    recordE_text[i] = process(recordE_text[i])
for i in tqdm(range(len(recordE_text))):
    recordE_text[i] = " ".join(recordE_text[i])

100%|██████████| 13487/13487 [00:05<00:00, 2484.59it/s]


In [14]:
recordE_text

0        Abstract of Disbursements ac of Schools . From...
1        Disbursments account of Colored Schools month ...
2        Disbursements account of Colored Schools Jan. ...
3                     School Expenses Month of February 67
4        James F. Carlen Co. 5 keys , Nails , School Ho...
                               ...                        
13482    SubAssistant Commissioners Agents Monthly Repo...
13483    Educational . WARRENTON FEMALE COLLEGE , WARRE...
13484    G , JANUARY 29 , 186 Educational . WARRENTON F...
13485    Painting water colors , 10.00 Drawing , Embroi...
13486    THE NATIONAL ARCHIVES OF THE UNITED STATES War...
Name: transcription_text, Length: 13487, dtype: object

In [15]:
recordE_text[7000]

'1 . Amount of Tuition paid month , 3.50100. 2 . Remarks . Every thing Our Schools , progressive , encourageing .'

In [16]:
!pip3 install Cython
!pip install transformers
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 30.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.0-py

In [17]:
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-27 19:24:35,885 loading file /root/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1


Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

2022-06-27 19:25:03,256 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [18]:
from flair.models import SequenceTagger

tagger1 = SequenceTagger.load("flair/ner-english")

Downloading:   0%|          | 0.00/432M [00:00<?, ?B/s]

2022-06-27 19:25:17,734 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-06-27 19:25:19,582 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [19]:
from flair.models import SequenceTagger

tagger2 = SequenceTagger.load("flair/ner-english-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-27 19:26:41,796 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
2022-06-27 19:26:59,880 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


# ontonotes

In [20]:
lst=[]
label=[]
Per=[]
Date=[]
GPE=[]
ORG=[]
LOC=[]
Bulding=[]
Cardinal=[]
from flair.data import Sentence

recordE_onto=pd.DataFrame({"recordE_text":"", "PERSON-ontonotes":[], "GPE-ontonotes":"","ORG-ontonotes":"", "Date-ontonotes":""})
for i in range(7668):
        sentencess =Sentence(recordE_text[i])
        recordE_onto.loc[i,"recordE_text"]=recordE_text[i]
        tagger.predict(sentencess)
        dc={}
        P=[]
        D=[]
        G=[]
        L=[]
        O=[]
        B=[]
        C=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.97) & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PERSON":
                      P.append(j)
                elif dc[j]=="ORG":
                      O.append(j)
                elif dc[j]=="DATE":
                      D.append(j)
                elif dc[j]=="GPE":
                      G.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
                elif dc[j]=="FAC":
                      B.append(j)
                elif dc[j]=="CARDINAL":
                      C.append(j)
        Per.append(P)
        Date.append(D)
        ORG.append(O)
        GPE.append(G)
        LOC.append(L)
        Bulding.append(B)
        Cardinal.append(C)
        
            
recordE_onto["Date-ontonotes"]=Date
recordE_onto["PERSON-ontonotes"]=Per
recordE_onto["GPE-ontonotes"]=GPE
recordE_onto["ORG-ontonotes"]=ORG 
recordE_onto["LOC-ontonotes"]=LOC 
recordE_onto["Building-ontonotes"]=Bulding
recordE_onto["Cardinal-ontonotes"]=Cardinal

2022-06-27 19:27:33,192 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-27 19:28:04,880 Warning: An empty Sentence was created! Are there empty strings in your dataset?


In [21]:
recordE_onto["recordE_text"]=recordE_text

In [22]:
recordE_onto.head()

,recordE_text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,LOC-ontonotes,Building-ontonotes,Cardinal-ontonotes
0,Abstract of Disbursements ac of Schools . From...,[],[],[],"[Jany67 July , 1868]",[],[],[]
1,Disbursments account of Colored Schools month ...,[],[],[Colored Schools],[January 1867],[],[],[]
2,Disbursements account of Colored Schools Jan. ...,[],[],[Colored Schools],[Jan. 1867],[],[],[]
3,School Expenses Month of February 67,[],[],[],[Month of February 67],[],[],[]
4,"James F. Carlen Co. 5 keys , Nails , School Ho...","[JW Rumsey, Robert Young]",[],[James F. Carlen Co .],[],[],[Carrol Hall],"[5, 37.50, 20.00, 1, 8.71, 17th0, 15.00, 150.00]"


# Post processing for dates

In [23]:
recordE_onto["Date-ontonotes"]

0                                    [Jany67 July , 1868]
1                                          [January 1867]
2                                             [Jan. 1867]
3                                  [Month of February 67]
4                                                      []
                              ...                        
7663                                 [month of July 1869]
7664    [THE MONTH ENDING July 30th , 1869, 16 Years o...
7665      [Sunday, Saturday, School month, 25th of month]
7666    [month, Eighteen days, four days, two days, th...
7667                           [THE MONTH OF August 1869]
Name: Date-ontonotes, Length: 7668, dtype: object

In [24]:
G=[]
for i in recordE_onto["Date-ontonotes"]:
    M=[]
    for j in i:
        date=re.search(r'^\d\.*.*\d\d$' ,j)
        date2=re.search(r'^[A-Z]\.*.*\d\.*.*\d\d$' ,j)
        M.append(date)
        M.append(date2)
    G.append(M)
    

GF=[]
for i in G:
    Final_date=[]
    for j in i:
        if j!=None:
            Final_date.append(j.group())
    GF.append(Final_date)
    
            
      

In [25]:
len(GF)

7668

In [26]:
recordE_onto["postDate-ontonotes"]=GF

In [27]:
G=[]
for i in recordE_onto["postDate-ontonotes"]:
    M=[]
    for j in i:
        Final_date1=re.sub(r'([0-9]{1,2})\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        Final_date2=re.sub(r'([A-Z]\.*.*)\s([0-9]{1,2})\.*.*\s([0-9]{2,4})',r'\2-\1-\3' ,j)
        final_date3=re.sub(r'([A-Z][a-z]\S*)\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        final_date4=re.sub(r'(^[A-Za-z]\S*.*.)day\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        if Final_date1 != j:   
            M.append(Final_date1)
        if Final_date2 != j :
            M.append(Final_date2)
        if final_date3 != j :
            M.append(final_date3)
        if final_date4 != j :
            M.append(final_date4)
    G.append(M)
    

recordE_onto["postDate-ontonotes"]=G

# standardizing dates

In [28]:
NG=[]
for i in G:
    ng=[]
    for j in i:
        
        date=re.findall(r'([0-9]{1,2})-([A-Z][a-z]\S*)-([0-9]{2,4})',j)
        if date !=[]:
            dates=[]
            for k in date:
                for l in k:
                    dates.append(l)
                
                
                if dates[1] in ["August","Aug","augt","AUGUST","Augst","Augt","AUG","Augusta"]:
                    dates[1]= "August"
                elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER",'Sepr','Septh']:
                    dates[1]= "Septembre"
                elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                    dates[1]= "October"
                elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER", "The"]:
                    dates[1]= "November"
                elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber"]:
                    dates[1]= "December"
                elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY"]:
                    dates[1]= "January"
                elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY"]:
                    dates[1]= "February"
                elif dates[1] in ["March","Mar"]:
                    dates[1]= "March"
                elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                    dates[1]= "April"
                elif dates[1] in ["May","MAY"]:
                    dates[1]= "May"
                elif dates[1] in ["JUN","Jun","June"]:
                    dates[1]= "June"
                elif dates[1] in ["JULY","July","JUL"]:
                    dates[1]= "July"
                j = dates[0]+"-"+dates[1]+"-"+dates[2]
                ng.append(j)
        else :
                date = re.findall(r'(^[A-Za-z]\S*.*.)-([A-Z][a-z]\S*)-([0-9]{2,4})',j ) 
                dates=[]
                for k in date:
                    for l in k:
                        dates.append(l)
                
                
                    if dates[1] in ["August","Aug","augt","AUGUST","Augst",'Augt',"AUG","Augusta"]:
                        dates[1]= "August"
                    elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER","Sepr","Septh", "This", "This "]:
                        dates[1]= "Septembre"
                    elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                        dates[1]= "October"
                    elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER"]:
                        dates[1]= "November"
                    elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber"]:
                        dates[1]= "December"
                    elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY"]:
                        dates[1]= "January"
                    elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY"]:
                        dates[1]= "February"
                    elif dates[1] in ["March","Mar"]:
                        dates[1]= "March"
                    elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                        dates[1]= "April"
                    elif dates[1] in ["May","MAY"]:
                        dates[1]= "May"
                    elif dates[1] in ["JUN","Jun","June"]:
                        dates[1]= "June"
                    elif dates[1] in ["JULY","July","JUL"]:
                        dates[1]= "July"
                    if dates[0]== "fifteenth" :
                        dates[0]="15"
                    elif (dates[0]=="sixteenth") |(dates[0]=="This 16 "):
                        dates[0]="16"
                    elif dates[0]=="Twenty":
                        dates[0]="20" 
                    elif dates[0]=="Twentyseventh":
                        dates[0]="27" 
                    elif (dates[0]=="Twenty Eighth") |(dates[0]=="TwentyEight") |(dates[0]=="TwentyEight ") |(dates[0]=="Twentyeight"):
                        dates[0]="28"
                    elif (dates[0]=="Ninth") |(dates[0]=="Ninth ")|(dates[0]=="Augusta 9th ") |(dates[0]=="ninth") |(dates[0]=="ninth ")|(dates[0]=="Augusta 9th"):
                        dates[0]="9"
                    
                    elif dates[0]=='Seventh':
                        dates[0]="7" 
                    elif (dates[0]=='The') |(dates[0]=="The 9th") :
                        dates[0]="9"
                    elif dates[0]=="the 9" :
                        dates[0]="9"
                    elif (dates[0]=="Eighth ") |(dates[0]=="Eighth") |(dates[0]=="eighth") |(dates[0]=="eighth "):
                        dates[0]="8"
                    elif dates[0]=="Seventh ":
                        dates[0]="7"
                    elif (dates[0]=="Twenty ninth ")| (dates[0]=="Twenty ninth") | (dates[0]=="twenty ninth ")|(dates[0]=="Twenty Ninth ")|(dates[0]=="Twenty Ninth") :
                        dates[0]="29"
                    elif (dates[0]=="Thirty") |(dates[0]=="Thirty ") | (dates[0]=="thirtieth "):
                        dates[0]="30"
                    elif (dates[0]=="Thirty first ") |(dates[0]=="Thirty first"):
                        dates[0]="31"
                  
                    
                    elif dates[0]=="first ":
                        dates[0]="1"
                    elif (dates[0]=="Second ") |(dates[0]=="Second") |(dates[0]=="second")|(dates[0]=="second "):
                        dates[0]="2"
                    elif dates[0]=="fourth ":
                        dates[0]="4"
                    elif dates[0]=="tenth ":
                        dates[0]="10"
                    elif (dates[0]=="Fifth ")|(dates[0]=="Fifth") |(dates[0]=="fifth ")|(dates[0]=="fifth"):
                        dates[0]="5"
                    elif (dates[0]=="This") | (dates[0]=="This 7th ")|(dates[0]=="seventh ") |(dates[0]=="seventh") |(dates[0]=="Seventh "):
                        dates[0]="7"
                    elif (dates[0]=="This 31st") |(dates[0]=="Thirty-first "):
                        dates[0]="31"
                    elif (dates[0]=="Thirtieth") |(dates[0]=="Thirtieth ") |(dates[0]=="Thirty"):
                        dates[0]="30"
                    elif dates[0]=="twenty fifth ":
                        dates[0]="25"
                    elif dates[0]=="Twenty seventh ":
                        dates[0]="27"
                    elif (dates[0]=="Twenty sixth ") |(dates[0]=="Twentysixth ")|(dates[0]=="Twentysixth") |(dates[0]=="twentysixth "):
                        dates[0]="26"
                    elif dates[0]=="third ":
                        dates[0]="3" 
                    elif dates[0]=="third":
                        dates[0]="3"
                    elif dates[0]=="Third ":
                        dates[0]="3"
                    elif dates[0]=="Third":
                        dates[0]="3"
                    
                    elif dates[0]=="Tenth":
                        dates[0]="10" 
                    elif dates[0]=="Augusta":
                        dates[0]="9" 
                    elif (dates[0]=="this_sixth ")|(dates[0]=="sixth ")| (dates[0]=="Sixth") | (dates[0]=="Sixth ") |(dates[0]=="this sixth "):
                        dates[0]="6" 
                        
                    elif dates[0]=="Tenth ":
                        dates[0]="10"
                    elif (dates[0]=="twentieth")|(dates[0]=="This 20th "):
                        dates[0]="20"
                    elif (dates[0]=="twenty fourth ") |(dates[0]=="twenty fourth") |(dates[0]=="Twenty fourth ") |(dates[0]=="Twenty fourth"):
                        dates[0]="24"
                    elif (dates[0]=="fifteenth ") |(dates[0]=="Fifteenth ") |(dates[0]=="Fifteenth"):
                        dates[0]="15"
                    elif dates[0]=="twentieth ":
                        dates[0]="20"
                    elif dates[0]=="Eighteenth ":
                        dates[0]="18"
                    elif dates[0]=="Eighteenth":
                        dates[0]="18"
                    elif (dates[0]=="Nineteenth ") |(dates[0]=="Nineteenth"):
                        dates[0]="19"
                    elif dates[0]=="First ":
                        dates[0]="1"
                    elif dates[0]=="First":
                        dates[0]="1"
                    elif (dates[0]=="Sixteenth ")|(dates[0]=="this 16") |(dates[0]=="Sixteenth"):
                        dates[0]="16"
                   
                    elif dates[0]=="twenty third ":
                        dates[0]="23"
                    elif dates[0]=="twenty third":
                        dates[0]="23"
                    elif dates[0]=="Twenty third ":
                        dates[0]="23"
                    elif dates[0]=="Twenty Second ":
                        dates[0]="22"
                    elif dates[0]=="Twenty Second":
                        dates[0]="22"
                    elif dates[0]=="Twenty Eighth ":
                        dates[0]="28"
                    elif dates[0]=="Twentysenventh ":
                        dates[0]="27"
                    elif dates[0]=="twenty third ":
                        dates[0]="23"
                    elif (dates[0]=="twenty third") |(dates[0]=="Twentythird")|(dates[0]=="Twentythird "):
                        dates[0]="23"
                    elif (dates[0]=="fourteenth ") | (dates[0]=="fourteenth") | (dates[0]=="Fourteenth") | (dates[0]=="Fourteenth "):
                        dates[0]="14"
                    elif (dates[0]=="Eleventh ") | (dates[0]=="Eleventh") |(dates[0]=="eleventh ") :
                        dates[0]="11"
                    elif (dates[0]=="Thirteenth") | (dates[0]=="Thirteenth ") :
                        dates[0]="13"
                    elif (dates[0]=="seventeenth ") |(dates[0]=="seventeenth") |(dates[0]=="Seventeenth ") :
                        dates[0]="17"
                     
                     
                     
                    
                        
                    j = dates[0]+"-"+dates[1]+"-"+dates[2]
                    ng.append(j)
            
            
    NG.append(ng)   

In [29]:
recordE_onto["standard_date-ontonotes"]=NG

In [30]:
recordE_onto.to_csv("recordE_onto.csv",index=False)

# Ner english large 

In [31]:
lst=[]
label=[]
Per=[]
ORG=[]
LOC=[]
Other_name=[]
from flair.data import Sentence

recordE_large=pd.DataFrame({"recordE_text":"", "PERSON-nerenglarge":[], "LOC-nerenglarge":"","ORG-nerenglarge":""})
for i in range(7668):
        sentencess =Sentence(recordE_text[i])
        recordE_large.loc[i,"recordE_text"]=recordE_text[i]
        tagger2.predict(sentencess)
        dc={}
        P=[]
        L=[]
        G=[]
        O=[]
        Other=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") :
                      O.append(j)
                elif  (dc[j]=="MISC") :
                      Other.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
        Other_name.append(Other)
            

recordE_large["PERSON-nerenglarge"]=Per
recordE_large["LOC-nerenglarge"]=LOC
recordE_large["ORG-nerenglarge"]=ORG    
recordE_large["other_namelarge"] =Other_name 

2022-06-27 19:35:05,264 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-27 19:35:36,735 Warning: An empty Sentence was created! Are there empty strings in your dataset?


In [32]:
recordE_large.head()

,recordE_text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge,other_namelarge
0,Abstract of Disbursements ac of Schools . From...,[],[],[],[]
1,Disbursments account of Colored Schools month ...,[],[],[Colored Schools],[]
2,Disbursements account of Colored Schools Jan. ...,"[A, James Morrison, John Kimball, R Young, Joh...",[Va],"[Colored Schools, Carpenters Work, Carroll Hal...",[]
3,School Expenses Month of February 67,[],[],[],[]
4,"James F. Carlen Co. 5 keys , Nails , School Ho...","[JW Rumsey, H.H . Tulley, J.S, Robert Young]","[Carrol Hall, Jno Bligh,40.00]","[James F. Carlen Co, School House]",[]


In [33]:
recordE_large.to_csv("recordE_large.csv",index=False)

# Ner english

In [34]:
lst=[]
label=[]
Per=[]
ORG=[]
LOC=[]
Other_name=[]
from flair.data import Sentence

recordE_ner=pd.DataFrame({"recordE_text":"", "PERSON-nerenglish":"","LOC-nerenglish":"","ORG-nerenglish":[]})
for i in range(20):
        sentencess =Sentence(recordE_text[i])
        recordE_ner.loc[i,"recorde_text"]=recordE_text[i]
        tagger1.predict(sentencess)
        dc={}
        P=[]
        L=[]
        O=[]
        Other=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG")  :
                      O.append(j)
                elif  (dc[j]=="MISC") :
                      Other.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
        Other_name.append(Other)
            

recordE_ner["PERSON-nerenglish"]=Per
recordE_ner["LOC-nerenglish"]=LOC
recordE_ner["ORG-nerenglish"]=ORG 
recordE_ner["Other_name-nerenglish"]=Other_name 

In [35]:
recordE_ner.head()

,recordE_text,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish,recorde_text,Other_name-nerenglish
0,NaN,[],[],[],Abstract of Disbursements ac of Schools . From...,[]
1,NaN,[],[],[],Disbursments account of Colored Schools month ...,[]
2,NaN,"[James Morrison, John Kimball, Mohun Son Lumber]",[],[],Disbursements account of Colored Schools Jan. ...,[]
3,NaN,[],[],[],School Expenses Month of February 67,[]
4,NaN,[James F. Carlen Co],[],[],"James F. Carlen Co. 5 keys , Nails , School Ho...",[]


In [36]:
recordE_ner.to_csv("recordE_ner.csv",index=False)

In [37]:
recordE_onto["PERSON-nerenglarge"]=recordE_large["PERSON-nerenglarge"]
recordE_onto["LOC-nerenglarge"]=recordE_large["LOC-nerenglarge"]
recordE_onto["ORG-nerenglarge"]=recordE_large["ORG-nerenglarge"]
recordE_onto["other_namelarge"]=recordE_large["other_namelarge"]
recordE_onto["PERSON-nerenglish"]=recordE_ner["PERSON-nerenglish"]
recordE_onto["LOC-nerenglish"]=recordE_ner["LOC-nerenglish"]
recordE_onto["ORG-nerenglish"]=recordE_ner["ORG-nerenglish"]
recordE_onto["Other_name-nerenglish"]=recordE_ner["Other_name-nerenglish"]

In [38]:
to_compare_result_recordE=recordE_onto

In [39]:
to_compare_result_recordE.to_csv("to_compare_result_recordE.csv", index=False)